# Interacting with the Miri Pixel DB with Jupyter Notebook

In [ ]:
""" Import packages"""
from miridb import init_db, cancel_active_queries, delete_table, load_miri_tables, load_engine, enter_psql_command, get_size_of_table
import keyring, os

In [ ]:
engine = load_engine()

In [ ]:
""" Get name of current database"""
psql_string = """SELECT current_database();"""
print(enter_psql_command(engine, psql_string))

""" List tables defined in database """
table_names = engine.table_names()
print(table_names)

In [ ]:
""" Load objects for connecting to the DB"""
session, base, connection, cursor = init_db(engine)

In [ ]:
""" Lines to load in table definions and create the tables in the DB"""
load_miri_tables(base) 
base.metadata.create_all()

In [ ]:
""" Lines to delete all the tables in DB - need to be in this order because of key constraints""" 
nuke_everything = True
if nuke_everything:
    delete_table('correctedgroups', password, base)
    delete_table('groups', password, base)
    delete_table('correctedramps', password, base)
    delete_table('ramps', password, base)
    delete_table('correctedexposures', password, base)
    delete_table('exposures', password, base)
    delete_table('detectors', password, base)
    delete_table('pixels', password, base)

In [ ]:
""" Get size of the miri_pixel_db"""
psql_string = 'SELECT pg_size_pretty( pg_database_size(\'miri_pixel_db\') )'
print(enter_psql_command(engine, psql_string))
dict(zip(table_names,[get_size_of_table(engine, table_name) for table_name in table_names]))

In [ ]:
### View long running queries
### The first returned column is the process id, the second is duration, following the query and state of this activity.
### If state is idle you don’t need to worry about it, but active queries may be the reason behind low performances on your database.
psql_string = """SELECT
      pid,
      now() - pg_stat_activity.query_start AS duration,
      query,
      state
    FROM pg_stat_activity
    WHERE (now() - pg_stat_activity.query_start) > interval '20 seconds';"""
out = enter_psql_command(engine, psql_string)
print(out)

In [ ]:
""" Cancel any active queries that are stuck"""
cancel_active_queries(engine)

In [ ]:
### kill different states
state = 'idle in transaction'
psql_string = """SELECT pg_terminate_backend(pid) 
FROM pg_stat_activity 
WHERE datname= 'miri_pixel_db'
  AND state = \'""" + state + '\''
enter_psql_command(engine, psql_string)

## Selecting Files from JPL8 to Insert into DB

In [ ]:
import glob
import numpy as np
import os
from exposuresdb import complement
import itertools

In [ ]:
jpl8_dir = '***********' ### get this directory location from developers
data_dirs = [
 jpl8_dir + '09_Mode_Switch_no3_pt2/',
 jpl8_dir + '10_Long_Pers/',
 #jpl8_dir + '11_All_Subarray/',
 jpl8_dir + '12_Anneals_pt2/']
all_fpm101_fits = [complement(glob.glob(data_dir+'*.fits'),glob.glob(data_dir+'*_LVL2.fits')) for data_dir in data_dirs]
complete_file_list = list(itertools.chain.from_iterable(all_fpm101_fits))

In [ ]:
print('Total number of raw exposures to add to DB: ',len(complete_file_list))
file_sizes_gb = [os.path.getsize(file)/(10**9) for file in complete_file_list]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
print('Size of raw exposure FITS files that will be added to DB: ',sum(file_sizes_gb), 'GB')

In [ ]:
min_file = complete_file_list[file_sizes_gb.index(min(file_sizes_gb))]
hdu = fits.open(min_file)
print(min_file)
# hdu[0].header

In [ ]:
""" Write list of LVL1 JPL8 FITS files to add to MiriDB"""
with open('../complete_file_list.txt', 'w') as f:
    for item in complete_file_list:
        f.write("%s\n" % item)

## Generate UML Diagram for MIRI Pixel DB

In [ ]:
import codecs
import  sadisplay #needs `pip install sadisplay`
from sqlalchemy import Table
import os
from miridb import load_engine, init_db

In [ ]:
engine = load_engine()
session, base, connection, cursor = init_db(engine)
table_names = engine.table_names() ### needs `engine` to be defined from first section above
all_tables = [Table(table_name,  base.metadata, autoload=True, autoload_with=engine) for table_name in table_names]

In [ ]:
desc = sadisplay.describe(
    all_tables,
    show_methods=True,
    show_properties=True,
    show_indexes=True,
)
with codecs.open('schema.dot', 'w', encoding='utf-8') as f:
    f.write(sadisplay.dot(desc))
! dot -Tpng schema.dot > ../miri_pixel_db_uml.png
os.remove('schema.dot')